In [1]:
from decoder_head.core import *
from fastai2.text.all import *

In [2]:
class PermuteEmbedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, padding_idx):
        super().__init__()
        
        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim
        self.padding_idx = padding_idx
        
        # to conform to nn.Embedding api
        self.max_norm=None
        self.norm_type=2.0
        self.scale_grad_by_freq=False
        self.sparse = False

        self.weight = nn.Parameter( torch.Tensor(num_embeddings, embedding_dim) )
        nn.init.kaiming_uniform_(self.weight)
        self.p = nn.Parameter( torch.eye(self.num_embeddings) )
        self.p.requires_grad = False
        
        self.reset_parameters()
        
    def forward(self, words):
        return F.embedding(words, self.p @ self.normalized_weight())
    
    def normalized_weight(self):
        w1 = self.weight / self.weight.norm(dim=1).unsqueeze(1)
        w2 = w1 - w1.mean(0)
        w3 = w2 / w2.norm(dim=1).unsqueeze(1)
        return w3
        
    def reset_parameters(self): pass

import decoder_head.core
decoder_head.core.PermuteEmbedding = PermuteEmbedding

In [3]:
vocab_en= make_vocab(pd.read_pickle('data/en-100_tok/counter.pkl'), max_vocab=4000)
vocab_es= make_vocab(pd.read_pickle('data/es-100_tok/counter.pkl'), max_vocab=4000)

In [4]:
path = 'data/en-100_tok/'
mult = 4
bs = 80
seq_len = 70

lm = DataBlock(blocks=(TextBlock(vocab=vocab_en, is_lm=True),),
                get_x=read_tokenized_file,
                get_items=partial(get_text_files, folders=['train', 'valid']),
                splitter=FuncSplitter(lambda itm: itm.parent.name == 'valid'))

dbunch_lm = lm.databunch(path, path=path, bs=bs, seq_len=seq_len)

In [5]:
learn = language_model_learner(
    dbunch_lm,
    pAWD_LSTM,
    opt_func=opt,
    pretrained=False,
    config=awd_lstm_lm_config,
    drop_mult=0.1,
    metrics=[accuracy, top_k_accuracy]
)

In [6]:
learn.load('emb_norm_rows_columns') # en LM

In [7]:
with open('data/en-es.txt') as f:
    en_es = f.readlines()
    
en_es = [l.strip() for l in en_es]

en_es_dict = defaultdict(list)

for l in en_es:
    source, target = l.split()
    en_es_dict[source].append(target)

In [9]:
len(en_es_dict)

93084

Check that we have the source word in the model trained on English

In [10]:
vocab_en_set = set(vocab_en)
en_es_dict = {k: v for k, v in en_es_dict.items() if k in vocab_en_set}

In [11]:
len(en_es_dict)

3475

Make sure we have the target word in Spanish

In [12]:
vocab_es_set = set(vocab_es)
en_es_dict = {k: vocab_es_set.intersection(set(v)) for k, v in en_es_dict.items() if vocab_es_set.intersection(set(v))}

In [13]:
len(en_es_dict)

2451

Attemp translation from English to Spanish

In [14]:
learn.model[0].encoder.weight.requires_grad

True

In [15]:
learn.optimize_permutation()

In [16]:
learn.model[0].encoder.weight.requires_grad

False

In [17]:
nn.init.kaiming_normal_(learn.model[0].encoder.p);

In [18]:
learn.model[0].encoder.weight.data = torch.load('data/embeddings_es.torch').data

In [19]:
learn.model[0].encoder.p.requires_grad

True

In [20]:
learn.model[0].encoder.weight.requires_grad

False

We now have an English LM with Spanish embeddings loaded. Let's see how we do on translation.

One thing worth keeping in mind are the missing English words from the Spanish vocabulary. This is an interesting situation and makes the task even harder.

In [21]:
learn.loss_func = lambda preds, targs: aza_loss(learn, preds, targs)

In [ ]:
learn.fit_one_cycle(1, 1e-3,  moms=(0.8, 0.7, 0.8), wd=1e-7)

epoch     train_loss  valid_loss  accuracy  top_k_accuracy  time    


In [ ]:
learn.fit_one_cycle(1, 1e-3,  moms=(0.8, 0.7, 0.8), wd=1e-7)

In [ ]:
# learn.save('pLSTM_emb_norm_en-es')

In [22]:
learn.load('pLSTM_emb_norm_en-es')

In [23]:
learn.validate()

(#3) [5.05198335647583,0.25118646025657654,0.4493016004562378]

In [37]:
def top_n_translation_acc(top_n=1):
    p = learn.model[0].encoder.p
    hits = 0
    for k, v in en_es_dict.items():
        idx_en = vocab_en.index(k)
        for vv in v:
            idx_es = vocab_es.index(vv)
            if idx_es in p[idx_en].argsort(descending=True)[:top_n]:
                hits += 1
#                 print(k, vv)
                break
    return hits/len(en_es_dict)

In [38]:
top_n_translation_acc(100)

0.031007751937984496

In [39]:
def top_n_translation_acc(top_n=1):
    p = learn.model[0].encoder.p
    hits = 0
    for k, v in en_es_dict.items():
        idx_en = vocab_en.index(k)
        for vv in v:
            idx_es = vocab_es.index(vv)
            if idx_es in p[idx_en].argsort(descending=True)[:top_n]:
                hits += 1
                print(k, [vocab_es[i] for i in p[idx_en].argsort(descending=True)[:5]])
                break
    return hits/len(en_es_dict)

In [27]:
en_es_dict['have']

{'han', 'tener', 'tienen'}

In [41]:
top_n_translation_acc(5)

food ['enlaces', 'comida', 'producción', 'plata', 'cobre']
illinois ['duró', 'catedral', 'vivienda', 'estreno', 'illinois']
looking ['paulo', 'lorenzo', 'aires', 'buscando', 'cobre']


0.0012239902080783353

In [29]:
def top_n_translation_acc(top_n=1):
    p = learn.model[0].encoder.p
    hits = 0
    for k, v in en_es_dict.items():
        idx_en = vocab_en.index(k)
        print(k, [vocab_es[i] for i in p[idx_en].argsort(descending=True)[:5]])
        for vv in v:
            idx_es = vocab_es.index(vv)
            if idx_es in p[idx_en].argsort(descending=True)[:top_n]:
                hits += 1
                break
    return hits/len(en_es_dict)

In [30]:
top_n_translation_acc(100)

the ['«', 'nacionalidad', 'creció', 'argentina', 'wisconsin']
and ['sin', 'nueva', 'obra', 'opinión', 'aunque']
was ['naciones', 'islas', 'estados', 'superar', 'de']
for ['cargo', 'elaboración', 'elección', 'sesión', 'publicación']
that ['o', 'oportunidades', 'negó', 'cargo', 'guerra']
with ['publicación', 'revista', 'cargo', 'gestión', 'elección']
from ['basados', 'casos', 'convirtiéndose', 'entre', 'posición']
this ['plena', 'cercana', 'alta', 'u', 'misión']
his ['unidad', 'fuente', 'unión', 'principal', 'reserva']
not ['bajo', 'minas', 'santa', 'clase', 'carne']
are ['xxunk', 'fuerza', 'unidad', 'minas', 'mujeres']
talk ['actividades', 'resto', 'favor', 'lucha', 'contexto']
which ['u', 'carta', 'expedición', 'mano', 'o']
also ['tribus', 'mujeres', 'renta', 'fuerzas', 'minas']
has ['plena', 'oposición', 'inteligencia', 'coalición', 'democracia']
were ['fuerza', 'renta', 'minas', 'islas', 'autoridad']
but ['búsqueda', 'derechos', 'guerra', 'opinión', 'carta']
have ['plena', 'renta', '

section ['yo', 'otro', 'socialista', 'guitarrista', 'b']
single ['pública', 'paulo', 'pan', 'dominicana', 'latina']
included ['elecciones', '1990', '1970', '1980', '1950']
home ['origen', 'música', 'educación', 'voz', 'finalmente']
women ['fin', 'hacia', 'y', 'en', 'sobre']
television ['xxmaj', 'moneda', ',', 'excepción', '(']
member ['otro', 'otra', 'uno', 'el', 'mexicana']
goals ['productos', 'alimentos', 'sangre', 'cobre', 'cerámica']
sources ['expresión', 'agrícola', 'figuras', 'intelectual', 'costumbres']
book ['simplemente', 'finalmente', 'mexicano', 'continúa', 'también']
station ['progreso', 'derecho', 'secundarios', 'allí', 'mantenimiento']
order ['adelante', 'cómo', 'atrás', 'vez', 'matrimonio']
old ['como', 'actualidad', 'figura', 'concepción', 'también']
information ['sociales', 'contexto', 'depende', 'entender', 'renacimiento']
set ['política', 'económico', 'humano', 'esclavos', 'sexual']
own ['—', ',', 'opera', '―', 'pesca']
text ['aire', 'piano', 'lo', 'canto', 'latín']


kingdom ['”', 'destino', 'actualmente', 'ahora', 'luego']
books ['•', 'orientación', 'antigüedad', 'fotografía', 'síntomas']
important ['las', 'cualquier', 'productos', 'comida', 'gas']
northern ['cabe', 'apareció', 'actualidad', 'sinopsis', 'protagonizada']
love ['a', 'principalmente', 'fundamentalmente', 'por', 'claramente']
canadian ['tal', 'buena', 'victoria', 'pocos', 'mismos']
press ['impacto', 'reconocimiento', 'daños', 'daño', 'conocimiento']
type ['hermana', 'madre', 'princesa', 'hermano', 'vasco']
act ['vive', 'portugués', 'española', 'luego', 'paulo']
editor ['venezuela', 'sinopsis', 'oficialmente', 'actuó', 'inicialmente']
schools ['áfrica', '•', 'capital', 'fiestas', 'orígenes']
program ['asociado', 'unido', 'león', 'progreso', 'empleo']
once ['cantidad', 'formas', 'expresión', 'venta', 'voces']
issue ['xv', 'universitaria', 'vasco', 'nº', 'mexicana']
social ['posee', 'presenta', 'se', 'es', 'son']
germany ['une', 'dirige', 'conocimiento', '”', 'destino']
production ['cult

technology ['granada', 'guion', 'pablo', 'taxonomía', 'martín']
future ['jr', 'música', 'surgió', 'localidad', 'originalmente']
formed ['1970', '1920', '1960', '1910', '1950']
census ['aragón', 'moderno', 'descrita', 'hollywood', '1996']
images ['’', 'miss', 'pop', 'b.', 'a.']
results ['esclavos', 'alimentos', 'áfrica', 'seguridad', 'pueblos']
return ['cómo', 'algunas', 'algunos', 'adelante', 'clases']
quality ['nacida', 'considerado', 'características', 'destaca', 'ahora']
construction ['mujeres', 'alfonso', 'c.', 'mitología', 'española']
zealand ['ofrece', 'dirige', 'reconocimiento', 'través', 'une']
front ['negro', 'mujeres', 'verde', 'princesa', 'letras']
cover ['n', '’', 'un', '12', '$']
model ['carmen', 'universo', 'inglés', 'man', 'baloncesto']
read ['variantes', 'inmediato', 'it', 'método', 'métodos']
material ['comportamiento', 'terror', 'salvador', 'comunicación', 'moda']
strong ['ayuda', 'amenaza', 'opción', 'fuente', 'esperanza']
coach ['ejerció', 'comúnmente', 'significa',

decision ['adelante', 'cómo', 'grado', 'cuidado', 'activo']
certain ['presenta', 'incluye', 'limita', 'opción', 'contiene']
minutes ['geografía', 'voz', 'trayectoria', 'importancia', 'cultura']
spain ['reconocimiento', 'dirige', 'culto', 'día', 'apellido']
plays ['time', 'animación', 'civil', 'house', 'vocal']
joseph ['actual', 'antigua', 'actualidad', 'crónica', 'derecha']
lord ['cabe', 'actualidad', 'será', 'antes', 'aparte']
decided ['adelante', 'cómo', 'clases', 'vez', 'poco']
operations ['medicina', 'electricidad', 'pobreza', 'salud', 'moda']
function ['man', 'ropa', 'company', 'negro', 'rey']
louis ['oficial', 'leyenda', 'histórica', 'dedicada', 'antiguo']
assembly ['próximo', 'sexo', 'fruto', '12', 'propietario']
queen ['acerca', 'del', 'dedicado', 'fin', 'sin']
security ['evolución', 'identidad', 'personajes', 'playa', 'naturaleza']
uses ['clase', 'art', 'producciones', 'asiáticos', 'civil']
ohio ['investigación', 'utilización', 'constitución', 'evolución', 'página']
operation 

...

positions ['principales', 'secundarios', 'visual', 'públicas', 'biología']
stay ['hasta', 'alrededor', 'bienes', 'pueblos', 'ríos']
titled ['únicamente', 'sólo', 'anteriormente', 'solamente', 'actuó']
combat ['composición', 'actividad', 'pintura', 'descrita', 'figura']
palace ['belleza', 'pasa', 'abogado', 'juan', 'obispo']
ordered ['formas', 'man', 'diversas', 'obligado', 'cierta']
opposition ['interpretación', 'cualquier', 'santiago', 'papel', 'mecanismo']
attempts ['adelante', 'clases', 'atrás', 'cómo', 'importancia']
wrestling ['ley', 'provincia', 'universidad', 'presidencia', 'billboard']
critics ['legado', '0.02', 'senador', 'patrimonio', 'raza']
growing ['griegos', 'negros', 'indiana', 'agrícola', 'laguna']
establish ['editor', 'art', 'publicaciones', 'revistas', 'universidades']
hands ['c.', 'letras', 'i.', 'ahora', '?']
participated ['órganos', 'dichos', 'pueblos', 'mercados', 'estas']
poetry ['belleza', 'ruta', 'historias', 'vicente', 'fotografía']
materials ['distribución', 

failure ['castellano', 'francés', 'clases', 'matrimonio', 'escritor']
strength ['latino', 'características', 'xv', 'portugués', 'idioma']
knight ['estudió', 'tomó', 'contrajo', 'fin', 'sentido']
hungary ['pablo', 'dibujo', 'cardenal', 'dirige', 'método']
keeping ['hacerse', '4', 'fundado', 'my', 'primero']
recordings ['territorial', 'fe', 'primavera', '*', 'jean']
functions ['verdad', 'love', 'media', 'voluntad', 'egipto']
calls ['hall', 'sebastián', 'entender', 'ahí', 'lucas']
criticism ['la', 'don', 'diversos', 'cabo', 'todo']
treaty ['nunca', 'niño', 'inglés', 'vive', 'yo']
selection ['aquel', 'cine', 'perú', 'material', 'esto']
colonial ['observación', 'serían', 'idea', 'fueron', 'triple']
motor ['diseñado', 'austria', 'formada', 'grecia', 'rosa']
obtained ['miami', 'king', '1922', '1965', 'max']
circuit ['bernardo', 'henry', 'comunicación', 'ocupó', 'jefferson']
determined ['propiedades', 'unas', 'cosas', 'usualmente', 'fuertemente']
fuel ['ayuda', 'presencia', 'esperanza', 'orden

represents ['civil', 'conferencias', 'políticos', 'prácticas', 'exposiciones']
franklin ['gobierno', 'calle', 'continuación', 'colaboración', 'renuncia']
viewers ['comportamiento', 'progreso', 'raíces', 'conservación', 'concepto']
differences ['dominio', 'régimen', 'áreas', 'sus', 'conceptos']
vocal ['iglesias', 'pueblos', 'referirse', 'playa', '[']
element ['fallecido', 'piano', 'atrás', 'través', 'xvi']
roof ['tv', 'canto', 'heredero', 'rico', 'piano']
colour ['xi', 'escribe', 'variedades', 'griego', 'utiliza']
prisoners ['verdadero', '…', '=', 'orientación', 'responsabilidad']
steam ['llevaba', 'animales', 'comparación', 'mamíferos', '4']
testing ['portugal', 'escribir', 'rica', 'religiosos', 'it']
territories ['colecciones', 'university', 'taxonomía', 'feria', 'común']
choose ['mariano', 'radical', 'reducción', 'rebelión', 'salamanca']
journey ['ritmo', 'derecho', 'reparto', 'humor', 'rosario']
costs ['tierra', 'park', 'juan', 'a.', 'esto']
soldier ['empleo', 'este', 'comunicacione

artwork ['luis', 'peter', 'rasgos', 'asesinado', 'duración']
occasions ['método', 'obispo', 'testamento', 'paisaje', 'hidalgo']
struggle ['realizando', 'morir', 'pedro', 'mundo', 'nicolás']
transition ['ojo', 'salida', 'herrera', 'contar', 'proteínas']
legacy ['arquitectura', 'real', 'cocina', 'radio', 'españolas']
denied ['ocupar', 'of', 'investigación', 'construcciones', 'secretaría']
relationships ['fallecido', 'programa', 'i.', 'embajador', 'primeros']
signing ['comedia', 'fallecido', 'literaria', 'xi', 'liderazgo']
believes ['escuelas', 'xxfake', 'tierras', 'puesta', 'utiliza']
reveals ['flores', 'colombia', 'funciones', 'células', 'suele']
defeating ['resultó', 'actuó', 'apareció', 'quedó', 'sucedió']
ranks ['cuestiones', 'plantas', 'gran', 'bautista', 'planta']
appointment ['jazz', 'b', 'video', 'primaria', 'privada']
sessions ['autonomía', 'retiro', 'conciencia', 'testamento', 'abogado']
trials ['francisco', 'trabajo', 'aragón', 'presidente', 'emperador']
mines ['servicios', 'r

0.031007751937984496

In [32]:
def top_n_translation_acc(top_n=1):
    p = learn.model[0].encoder.p
    learned_embeddings = learn.model[0].encoder.p @ learn.model[0].encoder.weight
    
    hits = 0
    for k, v in en_es_dict.items():
        idx_en = vocab_en.index(k)
        diff = learn.model[0].encoder.weight - learned_embeddings[idx_en]
        candidates = diff.sum(-1).argsort(descending=False)[:top_n]
        
        for vv in v:
            idx_es = vocab_es.index(vv)
            if idx_es in candidates:
                hits += 1
                break
    return hits/len(en_es_dict)

In [33]:
top_n_translation_acc(100)

0.03386372909016728

In [35]:
def top_n_translation_acc(top_n=1):
    p = learn.model[0].encoder.p
    learned_embeddings = learn.model[0].encoder.p @ learn.model[0].encoder.normalized_weight()
    
    hits = 0
    for k, v in en_es_dict.items():
        idx_en = vocab_en.index(k)
        diff = learn.model[0].encoder.normalized_weight() - learned_embeddings[idx_en]
        candidates = diff.sum(-1).argsort(descending=False)[:top_n]
        
        for vv in v:
            idx_es = vocab_es.index(vv)
            if idx_es in candidates:
                hits += 1
                break
    return hits/len(en_es_dict)

In [36]:
top_n_translation_acc(100)

0.03223174214606283